In [1]:
print(sc.version)

2.4.0-cdh6.3.0


In [3]:
# Record time
import time
from itertools import islice

start_time = time.time()

def tic():
    global start_time 
    start_time = time.time()

def tac():
    t_sec = round(time.time() - start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

In [ ]:
# Check file in kadochnikov file

In [4]:
! ls -l /project2/msca/kadochnikov/data 

total 2654944
-rwxr-xr-x 1 kadochnikov kadochnikov 702878193 Apr  9  2015 2007.csv
-rwxr-xr-x 1 kadochnikov kadochnikov 167562534 Apr  9  2015 2007.zip
-rwxr-xr-x 1 kadochnikov kadochnikov 689413344 Apr 10  2015 2008.csv
-rwxr-xr-x 1 kadochnikov kadochnikov 161037442 Apr 10  2015 2008.zip
drwxr-sr-x 2 kadochnikov kadochnikov      4096 Nov  4  2016 airbnb
drwxr-sr-x 2 kadochnikov kadochnikov      4096 Nov 12  2016 air_out
drwxr-sr-x 2 kadochnikov kadochnikov      4096 Nov  4  2016 air_travel
drwxr-sr-x 2 kadochnikov kadochnikov      4096 May 11  2018 austin
drwxr-sr-x 2 kadochnikov kadochnikov      4096 Oct  5  2018 books
-rwxr-xr-x 1 kadochnikov kadochnikov  26085508 Oct 31  2014 BX-CSV-Dump.zip
-rwxr-xr-x 1 kadochnikov kadochnikov 877080187 Apr 16  2015 caseload.csv
drwxr-sr-x 2 kadochnikov kadochnikov      4096 Sep 18  2019 emails
drwxr-sr-x 3 kadochnikov kadochnikov      4096 May  8  2015 examples-data
drwxr-sr-x 2 kadochnikov kadochnikov      4096 Oct 12 23:40 fasttext
drwxr-sr-x 2

# 1. Copy the file into your own directory on Linux server

In [5]:
! scp /project2/msca/kadochnikov/data/Municipal_Court_Caseload_Information.zip /project2/msca/jiangys/bdp/Municipal_Court_Caseload_Information.zip

In [6]:
# make sure file is in
! ls -l /project2/msca/jiangys/bdp

total 943584
drwxr-xr-x 2 jiangys jiangys      4096 Oct  8 22:55 cities
-rw-rw-r-- 1 jiangys jiangys 877080187 Apr 16  2015 Municipal_Court_Caseload_Information.csv
-rwxr-xr-x 1 jiangys jiangys  94212181 Oct 25 14:06 Municipal_Court_Caseload_Information.zip


# 2. Unzip the file

In [7]:
! unzip -o /project2/msca/jiangys/bdp/Municipal_Court_Caseload_Information.zip -d /project2/msca/jiangys/bdp/Municipal_Court_Caseload_Information.csv

Archive:  /project2/msca/jiangys/bdp/Municipal_Court_Caseload_Information.zip
checkdir:  cannot create extraction directory: /project2/msca/jiangys/bdp/Municipal_Court_Caseload_Information.csv
           File exists


In [14]:
# make sure file is in
! ls -l /project2/msca/jiangys/bdp

total 948576
drwxr-xr-x 2 jiangys jiangys      4096 Oct  8 22:55 cities
-rw-rw-r-- 1 jiangys jiangys 877080187 Apr 16  2015 Municipal_Court_Caseload_Information.csv
-rwxr-xr-x 1 jiangys jiangys  94212181 Oct 25 14:06 Municipal_Court_Caseload_Information.zip


# 3. Read the data into Spark RDD

In [28]:
! hadoop fs -mkdir /user/jiangys/bdp
! hadoop fs -put /project2/msca/jiangys/bdp/Municipal_Court_Caseload_Information.csv /user/jiangys/bdp/Municipal_Court_Caseload_Information.csv

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
mkdir: `/user/jiangys/bdp': File exists
Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0


In [29]:
! chmod o+x $HOME

In [30]:
tic()


readme_raw = sc.textFile("hdfs:///user/jiangys/bdp/Municipal_Court_Caseload_Information.csv")
headers = readme_raw.take(1)
readme = readme_raw.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter) #filter out header

tac()

Time passed: 0hour:0min:7sec


# 4. Ensure your process the header record correctly

In [31]:
readme.take(10)

['TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILED TO MAINTAIN FINANCIAL RESPONSIBILITY,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,22.40.00,FAILURE TO SIGNAL INTENT TO CHANGE LANES,8000 BLOCK RESEARCH,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,SPEEDING-STATE HIGHWAYS,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,20.00.00,NO SEAT BELT-DRIVER/PASSENGER,1000 BLOCK NORTH U S 183,N, ,N,N,Y',
 'TR,04/29/2010 07:00:00 AM +0000,23.45.00,SPEEDING - STATE HIGHWAYS - Less than 10% over limit,1000 BLOCK RESEARCH,N, ,N,N,Y',
 'PK,05/05/2010 07:00:00 AM +0000,15.28.00,PAY STATION RECEIPT NOT DISPLAYED,700 10TH ST W,N, ,N,N,Y',
 'PK,05/05/2010 07:00:00 AM +0000,15.23.00,EXPIRED PAY STATION RECEIPT,800 RIO GRANDE ST,N, ,N,N,Y',
 'PK,05/05/2010 07:00:00 AM +0000,15.22.00,EXPIRED PAY STATION RECEIPT,800 RIO GRANDE ST,N, ,N,N,Y',
 'PK,05/05/2010 07:00:00 AM +0000,15.18.00,EXPIRED PAY STATION RECEIPT,900 RIO GRANDE ST,N, ,N,N,Y',
 'PK,05/05/2010 0

In [32]:
readme.cache()

PythonRDD[14] at RDD at PythonRDD.scala:53

In [33]:
headers

['Offense Case Type,Offense Date,Offense Time,Offense Charge Description,Offense Street Name,Offense Cross Street Check , Offense Cross Street,School Zone,Construction Zone,Case Closed']

# 5. Calculate frequency of offenses by Offense Case Type

In [34]:
offense_case_type = readme.map(lambda s:s.split(",")[0])

In [35]:
offense_case_type = offense_case_type.map(lambda x: (x, 1))
frequency_by_offense_case_type = offense_case_type.reduceByKey(lambda x,y: x+y)

In [36]:
frequency_by_offense_case_type.take(10)

[('RL', 224188),
 ('PK', 3388981),
 ('CO', 240308),
 ('TR', 4313221),
 ('CM', 319078)]

# 6. Identify the most frequent offenses by Offense Charge Description (Show Offense Charge Description and offense frequency count in descending order)

In [37]:
offense_charge_desc = readme.map(lambda s:s.split(",")[3])

In [38]:
offense_charge_desc = offense_charge_desc.map(lambda x: (x, 1))
frequency_by_offense_charge_desc = offense_charge_desc.reduceByKey(lambda x,y: x+y)

In [39]:
frequency_by_offense_charge_desc.takeOrdered(1, lambda x:-x[1])

[('PAY STATION RECEIPT NOT DISPLAYED', 892013)]